In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn.neighbors as skn
import sklearn.ensemble as ske
import sklearn.metrics as skm
import sklearn.covariance as skc
import sklearn.svm as sks
import ensemble
from plotter import Plotter
import scipy.stats as ss

In [ ]:
pd.options.display.float_format = '{:,.3f}'.format

In [ ]:
def get_lof_optimal_k(x, y, k_min=2, k_max=50, plot=True):
    '''
    find optimal neighborhood size for LOF
    '''
    n = []
    a = []
    for n_neighbors in range(k_min, k_max+1):
        n.append(n_neighbors)
        lof = skn.LocalOutlierFactor(n_neighbors=n_neighbors)
        res = lof.fit(x)
        auc = skm.roc_auc_score(
            y_true=y,
            y_score=np.abs(res.negative_outlier_factor_)
        )
        a.append(auc)
        
    if plot:
        plt.plot(n,a)
        plt.xlabel('k')
        plt.ylabel('ROC AUC')
        plt.title('Optimal neighborhood size for LOF')
        plt.show()
        
    return n[np.argmax(a)]

In [ ]:
def get_mcd_optimal_contamination(
    x, y, contamination_min=0.05, contamination_max=0.5, contamination_steps=0.01,
    plot=True
):
    '''
    find optimal contamination for mcd
    '''
    val = []
    auc = []
    for contamination in np.arange(contamination_min, contamination_max, contamination_steps):
        mcd = skc.EllipticEnvelope(contamination=contamination)
        res = mcd.fit(x)
        a = skm.roc_auc_score(
            y_true=y,
            y_score=np.abs(res.score_samples(x))
        )
        auc.append(a)
        val.append(contamination)
        
    if plot:
        plt.plot(val,auc)
        plt.xlabel('contamination')
        plt.ylabel('ROC AUC')
        plt.title('Optimal contamination for MCD')
        plt.show()
        
    return val[np.argmax(auc)], auc[np.argmax(auc)]

In [ ]:
def get_iforest_optimal_contamination(
    x, y, contamination_min=0.05, contamination_max=0.5, contamination_steps=0.01,
    plot=True
):
    '''
    find optimal contamination for mcd
    '''
    val = []
    auc = []
  
    for contamination in np.arange(contamination_min, contamination_max, contamination_steps):
        iforest = ske.IsolationForest(n_jobs=-1, contamination=contamination)
        res = iforest.fit(x)
        a = skm.roc_auc_score(
            y_true=y,
            y_score=np.abs(iforest.score_samples(x))
        )
        auc.append(a)
        val.append(contamination)
        
    if plot:
        plt.plot(val,auc)
        plt.xlabel('contamination')
        plt.ylabel('ROC AUC')
        plt.title('Optimal contamination for Isolation forest')
        plt.show()
        
    return val[np.argmax(auc)], auc[np.argmax(auc)]

In [ ]:
# note: ROC AUC appears to fluctuate with changing the contamination factor
# for MCD and Isolation Forest, but the fluctuations do not seem to follow
# a meaningful trend. Therefore optimizing contamination for these
# detectors may not be very meaningful and will be skipped for now
# in the interest of making calculations faster

In [ ]:
def get_scores(df, varibales, label, plot=True):
    # lof
    k = get_lof_optimal_k(df[variables], df[label], plot=plot)
    print('using k = {} as the optimal neighbourhood size for LOF'.format(k))
    lof = skn.LocalOutlierFactor(n_neighbors=k)
    res = lof.fit(df[variables])
    df['lof'] = np.abs(res.negative_outlier_factor_)
    
    # isolation forest
    iforest = ske.IsolationForest(n_jobs=-1)
    res = iforest.fit(df[variables])
    df['iforest'] = np.abs(res.score_samples(df[variables]))
  
    # mcd
    mcd = skc.EllipticEnvelope(contamination=0.1)
    res = mcd.fit(df[variables])
    df['mcd'] = np.abs(res.score_samples(df[variables]))

    # one class svd
    ocsvm = sks.OneClassSVM()
    res = ocsvm.fit(df[variables])
    df['ocsvm'] = 1/res.score_samples(df[variables])
    
    return df

In [ ]:
def aggregate(df, score_cols, normalization=ensemble.Normalize().std_norm, copy=True):
    
    if copy:
        df = df.copy()
        
    for x in score_cols:            
        df[x+'_norm'] = normalization(df[x])
        
    score_cols = [x+'_norm' for x in score_cols]
               
    # aggregation
    df = ensemble.Ensemble().rank_avg_ensemble(df, score_cols)
    df['rank_avg_score'] = 1/df['rank_avg']
    df = ensemble.Ensemble().avg_ensemble(df, score_cols)
    df = ensemble.Ensemble().maxpool_ensemble(df, score_cols)
    #df = ensemble.Ensemble().thresholded_avg(df, score_cols)
    df = ensemble.Ensemble().threshold_pruned_avg_ensemble(df, score_cols)
    df = ensemble.Ensemble().top_k_pruned_avg_ensemble(df, score_cols)
    
    return df

In [ ]:
def calc_auc(df, cols, label, plot=True):
    auc_dict = {}
    for x in cols:
        if plot:
            auc_dict[x] = Plotter.plot_roc(df[label], df[x], title=x)
        else:
            auc_dict[x] = skm.roc_auc_score(df[label], df[x])
    return auc_dict

In [ ]:
def data_reader(data_fname):
    df = pd.read_csv(data_fname, header=None)
    df.columns = ['x'+str(x) for x in df.columns]

    df = df.rename(columns={'x{}'.format(len(df.columns)-1): 'y'})
    df['y'] = df['y'].apply(lambda x: 0 if x=='n' else 1)
    
    variables = [x for x in df.columns if 'x' in x]
    
    return df, variables

In [ ]:
# run

In [ ]:
from IPython.display import display

auc_ = {}
corrmat_ = {}
corrmat_reord_ = {}

for data_fname in [
    'aloi-unsupervised-ad.csv',
    'annthyroid-unsupervised-ad.csv',
    'breast-cancer-unsupervised-ad.csv',
    #'kdd99-unsupervised-ad.csv',
    'letter-unsupervised-ad.csv',
    'pen-global-unsupervised-ad.csv',
    'pen-local-unsupervised-ad.csv',
    'satellite-unsupervised-ad.csv',
    'shuttle-unsupervised-ad.csv',
    'speech-unsupervised-ad.csv'
]:
    print(data_fname)
    
    df, variables = data_reader(data_fname)
    df = get_scores(df, variables, 'y', plot=False)

    df[['lof', 'iforest', 'mcd', 'ocsvm']].hist(bins=50)
    plt.show()

    auc = calc_auc(df, ['lof', 'iforest', 'mcd', 'ocsvm'], 'y', plot=False)
    scores_auc = pd.DataFrame({'auc': auc})
    display(scores_auc)

    norm_dict = {
        'std_norm': ensemble.Normalize().std_norm,
        'thresholded_std_norm': ensemble.Normalize().thresholded_std_norm,
        'min_max_norm': ensemble.Normalize().minmax_norm
    }

    agg_cols = [
        'avg', 'maxpool', 'rank_avg_score', #'thresholded_avg',
        'threshold_pruned_avg', 'top_k_pruned_avg'
    ]

    res = pd.DataFrame()
    
    corrmat_[data_fname] = {}
    corrmat_reord_[data_fname] = {}

    for norm in norm_dict.keys():
        print(norm)
        df_agg = aggregate(
            df, ['lof', 'iforest', 'mcd', 'ocsvm'],
            normalization = norm_dict[norm]
        )

        corrmat, corrmat_reord = Plotter.clustered_hmap(df_agg, agg_cols)
        
        corrmat_[data_fname][norm] = corrmat
        corrmat_reord_[data_fname][norm] = corrmat_reord
        
        auc = calc_auc(df_agg, agg_cols, 'y', plot=False)

        if len(res):
            res = pd.concat([res, pd.DataFrame({norm: auc})], axis=1)
        else:
            res = pd.DataFrame({norm: auc})
    display(res)
    
    auc_[data_fname] = res
    
    
    print('\n------------------------\n')

In [ ]:
def fisher_transform(r):
    return np.arctanh(r)

def inv_fisher_transfrom(z):
    return np.tanh(z)

In [ ]:
m = []
for data_set in corrmat_:
    for normalization in corrmat_[data_set]:
        m.append(corrmat_[data_set][normalization])

In [ ]:
cols = m[0].columns
ind = m[0].index

In [ ]:
m = np.array(m)

In [ ]:
res = ss.ttest_1samp(fisher_transform(m), 0, axis=0)

In [ ]:
res.pvalue

In [ ]:
r = inv_fisher_transfrom(np.mean(fisher_transform(m), axis=0))
Plotter.cluster_mat(pd.DataFrame(r, index=ind, columns=cols))